    Убрать это в файл обязательно

    Необходимые библиотеки

In [457]:
import time
import datetime
import requests
import hmac
import hashlib
import numpy as np
import pandas as pd
import sqlalchemy
import threading
import matplotlib.pyplot as plt

In [458]:
pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 50)

    Функции к бирже

In [199]:
def hashing(query_string):
    #хеширование 
    return hmac.new(secret_key.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()

In [521]:
def get_server_time():
    url = "https://api.bybit.com/v5/market/time"
    current_time = int(time.time() * 1000)
    data = '{}'    
    sign = hashing(str(current_time) + api_key + '5000' + data)

    headers = {
        'X-BAPI-API-KEY': api_key,
        'X-BAPI-TIMESTAMP': str(current_time),
        'X-BAPI-SIGN': sign,
        'X-BAPI-RECV-WINDOW': str(5000)
    }

    response = requests.post(url=url, headers=headers, data=data)
    print(response.text)

    
def get_tickers(category='linear'):
    #тут уже есть много интересного, можно сравнивать за 1 проход и обновлять только необходимое
    url = f"https://api.bybit.com/v5/market/tickers?category={category}"
    response = requests.request("GET", url)
    #print(response.text)
    return response.json()['result']['list']
    
    
def get_kline(symbol, limit=30, start='', end='', category='linear', interval=1):
    #получить данные по торговле, по умолчанию последние 30 минутных бар
    url = f"https://api.bybit.com/v5/market/kline?category={category}&symbol={symbol}&interval={interval}&limit={limit}&start={start}&end={end}"

    response = requests.get(url=url) 
    
    #print(response.text)
    return response.json()['result']['list']
    '''   
        Response Parameters​

        Parameter	Type	Comments
        category	string	Product type
        symbol	string	Symbol name
        list	array	
        An string array of individual candle
        Sort in reverse by startTime
        > list[0]: startTime	string	Start time of the candle (ms)
        > list[1]: openPrice	string	Open price
        > list[2]: highPrice	string	Highest price
        > list[3]: lowPrice	string	Lowest price
        > list[4]: closePrice	string	Close price. Is the last traded price when the candle is not closed
        > list[5]: volume	string	Trade volume. Unit of contract: pieces of contract. Unit of spot: quantity of coins
        > list[6]: turnover	string	Turnover. Unit of figure: quantity of quota coin    
    '''   
def market_open_order(symbol, side, orderType, qty, category='linear'):
    #открыть ордер по рынку
    url = 'https://api.bybit.com/v5/order/create'
    current_time = int(time.time() * 1000)
    data = '{' + f'"symbol": "{symbol}", "side": "{side}", "orderType": "{orderType}", "qty": "{qty}", "category": "{category}"' + '}'    
    sign = hashing(str(current_time) + api_key + '5000' + data)

    headers = {
        'X-BAPI-API-KEY': api_key,
        'X-BAPI-TIMESTAMP': str(current_time),
        'X-BAPI-SIGN': sign,
        'X-BAPI-RECV-WINDOW': str(5000)
    }

    response = requests.post(url=url, headers=headers, data=data)
    print(response.text)
    
    
def limit_open_order(symbol, side, orderType, qty, price, category='linear'):
    url = 'https://api.bybit.com/v5/order/create'
    current_time = int(time.time() * 1000)
    data = '{' + f'"symbol": "{symbol}", "side": "{side}", "orderType": "{orderType}",  "price": "{price}", "qty": "{qty}", "category": "{category}"' + '}'    
    sign = hashing(str(current_time) + api_key + '5000' + data)

    headers = {
        'X-BAPI-API-KEY': api_key,
        'X-BAPI-TIMESTAMP': str(current_time),
        'X-BAPI-SIGN': sign,
        'X-BAPI-RECV-WINDOW': str(5000)
    }
    response = requests.post(url=url, headers=headers, data=data)
    print(response.text)


def cancel_order(symbol, category='linear'):
    url = 'https://api.bybit.com/v5/order/cancel-all'
    current_time = int(time.time() * 1000)
    data = '{' + f'"symbol": "{symbol}", "category": "{category}"' + '}'
    sign = hashing(str(current_time) + api_key + '5000' + data)

    headers = {
        'X-BAPI-API-KEY': api_key,
        'X-BAPI-TIMESTAMP': str(current_time),
        'X-BAPI-SIGN': sign,
        'X-BAPI-RECV-WINDOW': str(5000)
    }
    response = requests.post(url=url, headers=headers, data=data)
    print(response.text)
    
    
def convert_unix_to_datetime(unix_timestamp):
    return datetime.datetime.utcfromtimestamp(int(unix_timestamp))


def get_last_proof_open(ticker):
    #пересчет последних свечей 
    
    df = pd.DataFrame(get_kline(symbol=ticker, limit=20, interval=1))# ALPHAUSDT, start='1706380500000'
    df.columns = ['startTime_unix','openPrice','highPrice','lowPrice','closePrice','volume','turnove']
    df['middlePrice'] = df.apply(lambda x: (float(x.openPrice)+float(x.closePrice))/2.0, axis=1)
    df = df.sort_values('startTime_unix')
    df['diffMidPrice'] = df.middlePrice.diff()
 
    data = df.iloc[1:-1].diffMidPrice
    if df.iloc[-1].diffMidPrice>(np.mean(data)+2*np.std(data)):
        return True
    else:
        return False

    
def get_last_proof_close(ticker):
    #пересчет последних свечей 
    
    df = pd.DataFrame(get_kline(symbol=ticker, limit=20, interval=1))# ALPHAUSDT, end='1706383320000'
    df.columns = ['startTime_unix','openPrice','highPrice','lowPrice','closePrice','volume','turnove']
    df['middlePrice'] = df.apply(lambda x: (float(x.openPrice)+float(x.closePrice))/2.0, axis=1)
    df = df.sort_values('startTime_unix')
    df['diffMidPrice'] = df.middlePrice.diff()
 
    data = df.iloc[1:-1].diffMidPrice
    if df.iloc[-1].diffMidPrice<(np.mean(data)-2*np.std(data)):
        return True
    else:
        return False

    Тестовые прогоны функций

In [13]:
get_tickers()

[{'symbol': '1000000VINUUSDT',
  'lastPrice': '0.013306',
  'indexPrice': '0.013270',
  'markPrice': '0.013305',
  'prevPrice24h': '0.013708',
  'price24hPcnt': '-0.029325',
  'highPrice24h': '0.013906',
  'lowPrice24h': '0.012630',
  'prevPrice1h': '0.013467',
  'openInterest': '147421700',
  'openInterestValue': '1961445.72',
  'turnover24h': '708096.0161',
  'volume24h': '53120200.0000',
  'fundingRate': '0.0001',
  'nextFundingTime': '1706054400000',
  'predictedDeliveryPrice': '',
  'basisRate': '',
  'deliveryFeeRate': '',
  'deliveryTime': '0',
  'ask1Size': '700',
  'bid1Price': '0.013304',
  'ask1Price': '0.013306',
  'bid1Size': '1000',
  'basis': ''},
 {'symbol': '10000LADYSUSDT',
  'lastPrice': '0.0005051',
  'indexPrice': '0.0005053',
  'markPrice': '0.0005053',
  'prevPrice24h': '0.0005237',
  'price24hPcnt': '-0.035516',
  'highPrice24h': '0.0005383',
  'lowPrice24h': '0.0004718',
  'prevPrice1h': '0.0005056',
  'openInterest': '6482967000',
  'openInterestValue': '32758

____________________

    Функция Init, заполняем таблицу значениями, делаем 1 раз, потом обновление должно быть

______________________

In [523]:
df_tickers = pd.DataFrame(get_tickers(), dtype=float)

/var/folders/cm/zvk2ndt154z3tmhlg5c7g4ww0000gq/T/ipykernel_1081/185492397.py:1: FutureWarning: Could not cast to float64, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame', either all columns will be cast to that dtype, or a TypeError will be raised.
  df_tickers = pd.DataFrame(get_tickers(), dtype=float)


In [524]:
df_tickers

,symbol,lastPrice,indexPrice,markPrice,prevPrice24h,price24hPcnt,highPrice24h,lowPrice24h,prevPrice1h,openInterest,openInterestValue,turnover24h,volume24h,fundingRate,nextFundingTime,predictedDeliveryPrice,basisRate,deliveryFeeRate,deliveryTime,ask1Size,bid1Price,ask1Price,bid1Size,basis
0,1000000VINUUSDT,0.014951,0.014965,0.014957,0.014383,0.039491,0.015050,0.014232,0.014845,1.422422e+08,2127516.59,3.862113e+05,2.648040e+07,0.0001,1.706400e+12,,,,0.0,1900.00,0.014950,0.014957,1700.00,
1,10000LADYSUSDT,0.000565,0.000564,0.000564,0.000563,0.003377,0.000577,0.000537,0.000569,6.461081e+09,3644049.46,7.318644e+05,1.313800e+09,0.0001,1.706400e+12,,,,0.0,205300.00,0.000564,0.000565,56500.00,
2,10000NFTUSDT,0.004092,0.004091,0.004092,0.004260,-0.039436,0.004316,0.004082,0.004086,9.074922e+07,371345.81,3.874044e+05,9.330838e+07,-0.000067,1.706400e+12,,,,0.0,54800.00,0.004093,0.004094,27340.00,
3,10000SATSUSDT,0.004965,0.004960,0.004965,0.004840,0.025826,0.005350,0.004802,0.005042,1.343024e+09,6668116.15,5.848533e+07,1.151976e+10,0.0001,1.706400e+12,,,,0.0,119000.00,0.004965,0.004966,116500.00,
4,10000STARLUSDT,0.029990,0.029950,0.029990,0.029660,0.011126,0.031470,0.028910,0.030130,5.296044e+07,1588283.60,2.514905e+06,8.412308e+07,0.0001,1.706400e+12,,,,0.0,28150.00,0.029960,0.029970,5000.00,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,ZECUSDT,22.790000,22.770000,22.790000,22.640000,0.006625,22.910000,22.290000,22.820000,2.111212e+05,4811452.83,8.688943e+05,3.841590e+04,0.0001,1.706400e+12,,,,0.0,0.08,22.790000,22.810000,29.58,
324,ZENUSDT,8.114000,8.111000,8.114000,8.014000,0.012478,8.146000,7.911000,8.128000,2.396607e+05,1944606.92,9.418830e+05,1.172989e+05,0.0001,1.706400e+12,,,,0.0,3.70,8.113000,8.114000,1.30,
325,ZILUSDT,0.020830,0.020820,0.020830,0.020390,0.021579,0.020850,0.020240,0.020840,3.092001e+08,6440637.46,2.978998e+06,1.454487e+08,0.0001,1.706400e+12,,,,0.0,20270.00,0.020830,0.020840,6320.00,
326,ZKFUSDT,0.012970,0.012930,0.012970,0.012830,0.010911,0.013500,0.012380,0.013040,9.030038e+07,1171195.93,1.317198e+06,1.021891e+08,0.0001,1.706400e+12,,,,0.0,23350.00,0.012970,0.012980,8500.00,


In [525]:
#берем тех, у кого торговый объем больше 5млн
df_tickers = df_tickers[df_tickers['volume24h']>=5000000].sort_values('volume24h')

In [526]:
df_to_monitor = pd.DataFrame(columns=['symbol','mean','std','lvl','get_proof','in_pos'])

In [537]:
df_to_monitor['symbol'] = df_tickers['symbol']
df_to_monitor['get_proof'] = False
df_to_monitor['in_pos'] = False

In [528]:
#заполняем таблицу для бд
for ticker in list(df_to_monitor.symbol):
    #print(ticker)
    
    df = pd.DataFrame(get_kline(symbol=ticker, limit=20, interval=1))
    df.columns = ['startTime_unix','openPrice','highPrice','lowPrice','closePrice','volume','turnove']
    df['middlePrice'] = df.apply(lambda x: (float(x.openPrice)+float(x.closePrice))/2.0, axis=1)
    
    df_to_monitor.loc[df_to_monitor['symbol'].isin([ticker]),'mean'] = float(np.std(df.middlePrice))
    df_to_monitor.loc[df_to_monitor['symbol'].isin([ticker]),'std'] = float(np.mean(df.middlePrice))
    df_to_monitor.loc[df_to_monitor['symbol'].isin([ticker]),'lvl'] = float(np.mean(df.middlePrice)+1*np.std(df.middlePrice))

In [285]:
#df_to_monitor[['symbol','mean','std','lvl','sign']]

In [286]:
#df_to_monitor.loc[df_to_monitor['symbol'].isin(['USTCUSDT'])]

In [538]:
database_connection = sqlalchemy.create_engine(mysql_link)
df_to_monitor.to_sql(con=database_connection, name='tikers_linear_bybit_monitor', if_exists='replace', index=False)

-1

insert_encode_query = """INSERT INTO encoding_data
                            (id, face_encode)
                            VALUES (%s, %s)"""

    cursor = connection.cursor()
    cursor.executemany(insert_encode_query, detail_encoding)
    connection.commit()

__________________________________________________________

    Функция монитора каждую минуту

In [303]:
df_tickers = pd.DataFrame(get_tickers(), dtype=float)
df_tickers = df_tickers[df_tickers['volume24h']>=5000000].sort_values('volume24h')[['symbol','markPrice']]

/var/folders/cm/zvk2ndt154z3tmhlg5c7g4ww0000gq/T/ipykernel_1081/1683027070.py:1: FutureWarning: Could not cast to float64, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame', either all columns will be cast to that dtype, or a TypeError will be raised.
  df_tickers = pd.DataFrame(get_tickers(), dtype=float)


In [304]:
df_tickers

,symbol,markPrice
229,PENDLEUSDT,2.299200
107,DYDXUSDT,2.727000
267,SOLUSDT,93.040000
90,CROUSDT,0.081930
181,LOOKSUSDT,0.082510
241,RAREUSDT,0.102400
128,FLMUSDT,0.077630
13,1INCHUSDT,0.399000
242,RDNTUSDT,0.294100
284,TIAUSDT,17.236400


In [305]:
right = pd.DataFrame()
engine = sqlalchemy.create_engine(mysql_link)
with engine.connect() as con:

    rs = con.execute('select symbol, lvl, sign from tikers_linear_bybit_monitor where sign = 0')

    #for row in rs:
    #    print(row)
    right = pd.DataFrame(rs)#, dtype=float).copy()

In [306]:
right

,symbol,lvl,sign
0,PENDLEUSDT,2.296130,0
1,DYDXUSDT,2.726144,0
2,SOLUSDT,93.079306,0
3,CROUSDT,0.081989,0
4,LOOKSUSDT,0.082606,0
5,RAREUSDT,0.102495,0
6,FLMUSDT,0.077576,0
7,1INCHUSDT,0.399751,0
8,RDNTUSDT,0.294723,0
9,MANAUSDT,0.455202,0


In [307]:
df_tickers.columns

Index(['symbol', 'markPrice'], dtype='object')

In [308]:
right.columns

Index(['symbol', 'lvl', 'sign'], dtype='object')

In [309]:
df_tickers = df_tickers.merge(right, how='inner', on='symbol')

In [310]:
df_tickers

,symbol,markPrice,lvl,sign
0,PENDLEUSDT,2.299200,2.296130,0
1,DYDXUSDT,2.727000,2.726144,0
2,SOLUSDT,93.040000,93.079306,0
3,CROUSDT,0.081930,0.081989,0
4,LOOKSUSDT,0.082510,0.082606,0
5,RAREUSDT,0.102400,0.102495,0
6,FLMUSDT,0.077630,0.077576,0
7,1INCHUSDT,0.399000,0.399751,0
8,RDNTUSDT,0.294100,0.294723,0
9,TIAUSDT,17.236400,17.287256,0


In [311]:
df_tickers['sign'] = df_tickers[['markPrice','lvl']].apply(lambda x: 1 if x['markPrice']>x['lvl'] else 0, axis=1)
#> должно быть

In [325]:
df_tickers

,symbol,markPrice,lvl,sign
0,PENDLEUSDT,2.299200,2.296130,1
1,DYDXUSDT,2.727000,2.726144,1
2,SOLUSDT,93.040000,93.079306,0
3,CROUSDT,0.081930,0.081989,0
4,LOOKSUSDT,0.082510,0.082606,0
5,RAREUSDT,0.102400,0.102495,0
6,FLMUSDT,0.077630,0.077576,1
7,1INCHUSDT,0.399000,0.399751,0
8,RDNTUSDT,0.294100,0.294723,0
9,TIAUSDT,17.236400,17.287256,0


In [326]:
df_tickers[df_tickers.sign==1]#[0:4]

,symbol,markPrice,lvl,sign
0,PENDLEUSDT,2.299200,2.296130,1
1,DYDXUSDT,2.727000,2.726144,1
6,FLMUSDT,0.077630,0.077576,1
10,MANAUSDT,0.455500,0.455202,1
17,IDUSDT,0.288000,0.287890,1
18,AXLUSDT,0.939300,0.939298,1
19,STXUSDT,1.518970,1.516903,1
28,CHRUSDT,0.276000,0.275495,1
30,TRUUSDT,0.048130,0.048124,1
34,OPUSDT,3.082100,3.076223,1


In [327]:
tickers_lst = list(df_tickers[df_tickers.sign==1].symbol)#для теста первые 4 берем
#==0 должно быть 1

In [328]:
tickers_lst

['PENDLEUSDT',
 'DYDXUSDT',
 'FLMUSDT',
 'MANAUSDT',
 'IDUSDT',
 'AXLUSDT',
 'STXUSDT',
 'CHRUSDT',
 'TRUUSDT',
 'OPUSDT',
 'ARPAUSDT',
 'RVNUSDT',
 'API3USDT',
 'XLMUSDT',
 'ALGOUSDT',
 'COTIUSDT',
 'WOOUSDT',
 'SUNUSDT',
 '1000FLOKIUSDT',
 'BLZUSDT',
 'AIUSDT',
 'KASUSDT',
 'SWEATUSDT',
 'BIGTIMEUSDT',
 'MAVUSDT',
 'DUSKUSDT',
 'GRTUSDT',
 'TUSDT',
 'SUPERUSDT',
 'FTMUSDT',
 'IOTXUSDT',
 'MATICUSDT',
 'FLRUSDT',
 'ONEUSDT',
 'CTSIUSDT',
 'CKBUSDT',
 'TRXUSDT',
 'ASTRUSDT',
 'XRDUSDT',
 '10000STARLUSDT',
 'XRPUSDT',
 '1000RATSUSDT',
 'SEIUSDT',
 'ZILUSDT',
 'FITFIUSDT',
 'CHZUSDT',
 'GPTUSDT',
 'PEOPLEUSDT',
 'SHIB1000USDT',
 'GALAUSDT',
 'WSMUSDT',
 '10000LADYSUSDT',
 '1000BTTUSDT',
 '1000PEPEUSDT']

In [329]:
#берем инит только выборочно 

In [411]:
for ticker in tickers_lst:
    print(ticker)
    print(get_last_pruf(ticker))

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False


    Прогон по конкретному тикету

In [508]:
get_last_pruf_open('ALPHAUSDT')

False

    Как войти

In [404]:
df = pd.DataFrame(get_kline(symbol='ALPHAUSDT', limit=20, interval=1, start='1706380500000'))
df.columns = ['startTime_unix','openPrice','highPrice','lowPrice','closePrice','volume','turnove']
df['middlePrice'] = df.apply(lambda x: (float(x.openPrice)+float(x.closePrice))/2.0, axis=1)
df = df.sort_values('startTime_unix')
df['diffMidPrice'] = df.middlePrice.diff()

#df.iloc[:-1].middlePrice #до последнего значения
#df.iloc[-1:].middlePrice #последнее значение

data = df.iloc[1:-1].diffMidPrice
print(df.iloc[-1:].closePrice[0],df.iloc[-1:].diffMidPrice[0],np.mean(data)+2*np.std(data))

if df.iloc[-1:].diffMidPrice[0]>(np.mean(data)+2*np.std(data)):
    print(ticker)
    print(f'market_open_order: {df.iloc[-1:].middlePrice[0]}')

0.09815 0.00019999999999999185 6.390834885535014e-05
1000PEPEUSDT
market_open_order: 0.097965


    Как выйти

In [506]:
df = pd.DataFrame(get_kline(symbol='ALPHAUSDT', limit=20, interval=1, end='1706383320000'))#start 1706380500
df.columns = ['startTime_unix','openPrice','highPrice','lowPrice','closePrice','volume','turnove']
df['middlePrice'] = df.apply(lambda x: (float(x.openPrice)+float(x.closePrice))/2.0, axis=1)
df = df.sort_values('startTime_unix')
df['diffMidPrice'] = df.middlePrice.diff()

#df.iloc[:-1].middlePrice #до последнего значения
#df.iloc[-1:].middlePrice #последнее значение

data = df.iloc[1:-1].diffMidPrice
print(df.iloc[-1].closePrice,df.iloc[-1].diffMidPrice,np.mean(data)-2*np.std(data))

if df.iloc[-1].diffMidPrice<(np.mean(data)-2*np.std(data)):
    print('ALPHAUSDT')
    print(f'market_open_order: {df.iloc[-1].middlePrice}')

0.10313 -0.0028349999999999903 -0.0015216158852172162
ALPHAUSDT
market_open_order: 0.10405500000000001


In [504]:
df.iloc[1].closePrice

'0.09874'

In [505]:
df#.reset_index()

,startTime_unix,openPrice,highPrice,lowPrice,closePrice,volume,turnove,middlePrice,diffMidPrice
19,1706381640000,0.09778,0.09849,0.09778,0.09815,1426900,140064.45033,0.097965,NaN
18,1706381700000,0.09815,0.09876,0.09791,0.09874,387016,38015.78567,0.098445,0.000480
17,1706381760000,0.09874,0.09984,0.0986,0.09984,498953,49387.73672,0.099290,0.000845
16,1706381820000,0.09984,0.10087,0.09978,0.10081,709347,71002.27702,0.100325,0.001035
15,1706381880000,0.10081,0.10187,0.10056,0.10187,800927,80897.26635,0.101340,0.001015
14,1706381940000,0.10187,0.10481,0.10187,0.10477,684906,70581.61145,0.103320,0.001980
13,1706382000000,0.10477,0.10487,0.10373,0.1044,889160,92685.81504,0.104585,0.001265
12,1706382060000,0.1044,0.10529,0.10404,0.10427,726860,76118.21434,0.104335,-0.000250
11,1706382120000,0.10427,0.10495,0.104,0.10493,361904,37836.092,0.104600,0.000265
10,1706382180000,0.10493,0.10516,0.10464,0.10499,315602,33132.30685,0.104960,0.000360
